# Numerical Benchmarks

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from quantum_tools import *
from experimental import *
from spam import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

C:\Users\krist\Documents\ModelNISQ\notebooks\experiments\../../src_tf\spam.py:108: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if init is "random":
C:\Users\krist\Documents\ModelNISQ\notebooks\experiments\../../src_tf\spam.py:116: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif init is "ideal":
C:\Users\krist\Documents\ModelNISQ\notebooks\experiments\../../src_tf\spam.py:126: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if povm is "random":
C:\Users\krist\Documents\ModelNISQ\notebooks\experiments\../../src_tf\spam.py:140: SyntaxWarning: "is" with a literal. Did you mean "=="?
  elif povm is "ideal":


## Recover SPAM and Map

### Generate True Model, Full POVM and Inital

In [2]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n
rank = 16
c1 = 0.9
c2 = 0.9

#prep error and full POVM error
spam_target = SPAM(d=d)

init_target = c1*init_ideal(d) + (1-c1)*spam_target.init
povm_target = c2*povm_ideal(d) + (1-c2)*spam_target.povm

spam_target = SPAM(d=d,
                  init = init_target,
                  povm = povm_target)

kraus_target = DilutedKrausMap(
                               U=generate_unitary(d=d), 
                               c=0.1, 
                               d=d, 
                               rank=rank-1,
                               spam = spam_target
                              )

### Generate Synthetic Data with Noise

In [3]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

inputs_spam, _ = generate_pauliInput_circuits(n)

U_prep = inputs_spam
N_spam = U_prep.shape[0]

state = tf.repeat(spam_target.init[None,:,:], N_spam, axis=0)
state = apply_unitary(state, U_prep)
targets_spam = measurement(state, povm = spam_target.povm)

#add noise
targets_spam = add_noise_to_probs(targets_spam, 0.0316)

inputs_map, _ = generate_pauli_circuits(n = n, 
                                        circuit_target=None, 
                                        N = 2000-6**3, 
                                        trace=False)
U_prep, U_basis = inputs_map

N_map = U_prep.shape[0]
state = tf.repeat(tf.expand_dims(spam_target.init, axis=0), N_map, axis=0)
state = apply_unitary(state, U_prep)
state = kraus_target.apply_channel(state)
targets_map = measurement(state, U_basis, spam_target.povm)

#add noise
targets_map = add_noise_to_probs(targets_map, 0.0316)

### Fit Model, Initial and Corruption Matrix

In [4]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

spam_model = SPAM(d=d,
#                  use_corr_mat=True,
                  optimizer = tf.optimizers.Adam(learning_rate=0.01))

spam_model.pretrain(targets = [init_ideal(d), povm_ideal(d)],
                    num_iter = 300,
                    verbose = False,
                    )

spam_model.train(inputs = inputs_spam,
                 targets = targets_spam,
                 num_iter = 1000,
                 verbose = False,
                )

  0%|          | 0/300 [00:00<?, ?it/s]

6.335375165672933e-05


  0%|          | 0/1000 [00:00<?, ?it/s]

0.0031541816499257745


In [ ]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

model = ModelQuantumMap(channel = KrausMap(d = d, 
                                           rank = 64,
                                           spam = spam_model,
                                          ),
                        loss_function = ProbabilityMSE(),
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        logger = Logger(loss_function = channel_fidelity_loss),
                       )
                       

model.train(inputs = inputs_map,
            targets = targets_map,
            inputs_val = None,
            targets_val = [kraus_target],
            num_iter = 10000,
            N = 500,
            )

  0%|          | 0/10000 [00:00<?, ?it/s]

None -0.213161658897458
None -0.4840358415417085
None -0.6679935552035737
None -0.764831724635864
None -0.8180580030268604
None -0.8501190121550084
None -0.8705893950737538
None -0.8839405072881718
None -0.893383798682935
None -0.8988537581756438
None -0.9034298586394337
None -0.9073769571897835
None -0.9092631875594436
None -0.9108189242850416
None -0.9126693356946809
None -0.9131314132644448
None -0.9145833155688263
None -0.915295526266631
None -0.9163857791651008


In [ ]:
np.random.seed(43)
random.seed(43)
tf.random.set_seed(43)

model = ModelQuantumMap(channel = KrausMap(d = d, 
                                           rank = 32,
                                           spam = spam_model,
                                          ),
                        loss_function = ProbabilityMSE(),
                        optimizer = tf.optimizers.Adam(learning_rate=0.01),
                        logger = Logger(loss_function = channel_fidelity_loss),
                       )
                       

model.train(inputs = inputs_map,
            targets = targets_map,
            inputs_val = None,
            targets_val = [kraus_target],
            num_iter = 10000,
            N = 500,
            )